<a href="https://colab.research.google.com/github/rappjer1/geeup/blob/master/Running_MultiYears.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
!pip install earthengine-api
!earthengine authenticate

# Load the Drive helper and mount
from google.colab import drive
drive.mount('/content/drive')

# !pip install tensorflow==1.14
from __future__ import absolute_import, division, print_function, unicode_literals
import logging, re, keras, IPython, os, pprint
logging.getLogger("tensorflow").setLevel(logging.ERROR)

# # Install TensorFlow
# try:
#   # %tensorflow_version only exists in Colab.
#   %tensorflow_version 2.x
# except Exception:
#   pass
import matplotlib.pyplot as plt
import tensorflow as tf
import numpy as np
import datetime as dt
import os
from keras.callbacks import TensorBoard

%matplotlib inline
#tf.enable_eager_execution()
print(tf.__version__)

def parse_tfrecord(example_proto):
  """The parsing function.

  Read a serialized example into the structure defined by featuresDict.

  Args:
    example_proto: a serialized Example.
  
  Returns: 
    A tuple of the predictors dictionary and the label, cast to an `int32`.
  """
  parsed_features = tf.io.parse_single_example(example_proto, featuresDict)
  labels = parsed_features.pop(label)
  return parsed_features, tf.cast(labels, tf.int32)


# Keras requires inputs as a tuple.  Note that the inputs must be in the
# right shape.  Also note that to use the categorical_crossentropy loss,
# the label needs to be turned into a one-hot vector.
# need to figure out a clever way to introduce other data types
def toTuple(dict, label):
  return tf.transpose(list(dict.values())), tf.one_hot(indices=label, depth=nClasses)

from google.colab import auth
auth.authenticate_user()

project_id = 'amazing-sunset-256615'
!gcloud config set project {project_id}

if 'COLAB_TPU_ADDR' not in os.environ:
    print('ERROR: Not connected to a TPU runtime; please see the first cell in this notebook for instructions!')
else:
    tpu_address = 'grpc://' + os.environ['COLAB_TPU_ADDR']
    print ('TPU address is', tpu_address)

    with tf.Session(tpu_address) as session:
      devices = session.list_devices()

    print('TPU devices:')
    pprint.pprint(devices)

resolver = tf.contrib.cluster_resolver.TPUClusterResolver(tpu='grpc://' + os.environ['COLAB_TPU_ADDR'])
tf.contrib.distribute.initialize_tpu_system(resolver)
strategy = tf.contrib.distribute.TPUStrategy(resolver)


Running command using Cloud API.  Set --no-use_cloud_api to go back to using the API

/usr/local/lib/python3.6/dist-packages/IPython/utils/traitlets.py:5: UserWarning: IPython.utils.traitlets has moved to a top-level traitlets package.
  warn("IPython.utils.traitlets has moved to a top-level traitlets package.")
To authorize access needed by Earth Engine, open the following URL in a web browser and follow the instructions. If the web browser does not start automatically, please manually browse the URL below.

    https://accounts.google.com/o/oauth2/auth?client_id=517222506229-vsmmajv00ul0bs7p89v5m89qs8eb9359.apps.googleusercontent.com&scope=https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fearthengine+https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdevstorage.full_control&redirect_uri=urn%3Aietf%3Awg%3Aoauth%3A2.0%3Aoob&response_type=code

The authorization workflow will generate a code, which you should paste in the box below. 
Enter verification code: 4/twHXU_A359cIWCMkMF2UnJGSl1pb7aN_u4g7DKRMym3

Using TensorFlow backend.


1.15.0
Updated property [core/project].
TPU address is grpc://10.106.83.250:8470
TPU devices:
[_DeviceAttributes(/job:tpu_worker/replica:0/task:0/device:CPU:0, CPU, -1, 1632552742324286086),
 _DeviceAttributes(/job:tpu_worker/replica:0/task:0/device:XLA_CPU:0, XLA_CPU, 17179869184, 4448019903792033047),
 _DeviceAttributes(/job:tpu_worker/replica:0/task:0/device:TPU:0, TPU, 17179869184, 6069202339650114135),
 _DeviceAttributes(/job:tpu_worker/replica:0/task:0/device:TPU:1, TPU, 17179869184, 2967379594215618485),
 _DeviceAttributes(/job:tpu_worker/replica:0/task:0/device:TPU:2, TPU, 17179869184, 8904198647298172581),
 _DeviceAttributes(/job:tpu_worker/replica:0/task:0/device:TPU:3, TPU, 17179869184, 2562959025537348730),
 _DeviceAttributes(/job:tpu_worker/replica:0/task:0/device:TPU:4, TPU, 17179869184, 8070200644426126314),
 _DeviceAttributes(/job:tpu_worker/replica:0/task:0/device:TPU:5, TPU, 17179869184, 5169678392240035720),
 _DeviceAttributes(/job:tpu_worker/replica:0/task:0/device:

In [0]:
gdrive_folder = 'AIM_Data' 
train_desc = 'TF_train_Points'
test_desc = 'TF_test_Points'
fileNameSuffix = '.tfrecord.gz'
outputBucket = 'aim_nn'
trainFilePath = 'gs://' + outputBucket + '/' + train_desc + fileNameSuffix
testFilePath = 'gs://' + outputBucket + '/' + test_desc + fileNameSuffix

print(trainFilePath)
print(testFilePath)
print('Found training file.' if tf.io.gfile.exists(trainFilePath) 
    else 'No training file found.')

print('Found training file.' if tf.io.gfile.exists(testFilePath) 
    else 'No training file found.')

gs://aim_nn/TF_train_Points.tfrecord.gz
gs://aim_nn/TF_test_Points.tfrecord.gz
Found training file.
Found training file.


In [0]:
model = tf.keras.models.load_model('/content/drive/My Drive/test_model5_3000.h5')

In [0]:
def parse_image(example_proto):
  #example_proto = tf.cast(example_proto, dtype=tf.float32)
  return tf.io.parse_single_example(example_proto, imageFeaturesDict)

def parse_tfrecord(example_proto):
  """The parsing function.

  Read a serialized example into the structure defined by featuresDict.

  Args:
    example_proto: a serialized Example.
  
  Returns: 
    A tuple of the predictors dictionary and the label, cast to an `int32`.
  """
  parsed_features = tf.io.parse_single_example(example_proto, featuresDict)
  labels = parsed_features.pop(label)
  return parsed_features, tf.cast(labels, tf.int32)

years = ['2012','2017']
region = 'Nebraska'
bandNames = ['year','GDD_ss','pr', 'DOY','slop_mn', 'EVI_mx_14', 'EVI_r_14', 'EVI_mn_14', 
                          'GCVI_mx_14','GCVI_r_14', 'GCVI_mn_14', 
                          'NDVI_mx_14','NDVI_r_14', 'NDVI_mn_14', 
                          'NDWI_mx_14', 'NDWI_r_14', 'NDWI_mn_14',
			                    'ndwi_gi', 'grenArd','pdsi_nn', 'pr_ann', 
			                    'pr_erly', 'pr_grow', 'pds_grw', 'aridity',
                          'awc1000', 'agDvAwc',  'giDvAwc', 'pr_paw','g15DvAw','GCVI_15','WGI_p15','AGI_p15']
label = 'class'
featureNames = ['year','GDD_ss','pr', 'DOY','slop_mn', 'EVI_mx_14', 'EVI_r_14', 'EVI_mn_14', 
                          'GCVI_mx_14','GCVI_r_14', 'GCVI_mn_14', 
                          'NDVI_mx_14','NDVI_r_14', 'NDVI_mn_14', 
                          'NDWI_mx_14', 'NDWI_r_14', 'NDWI_mn_14',
			                    'ndwi_gi', 'grenArd','pdsi_nn', 'pr_ann', 
			                    'pr_erly', 'pr_grow', 'pds_grw', 'aridity',
                          'awc1000', 'agDvAwc',  'giDvAwc', 'pr_paw','g15DvAw','GCVI_15','WGI_p15','AGI_p15']



In [0]:
for year in years:
  import pprint
  filesList = !gsutil ls 'gs://'{outputBucket}
  imageFilePrefix = year +'_'+region+'_buffer_tfRecordne_right'
  # Get a list of all the files in the output bucket.
  print(filesList)
  # Get only the files generated by the image export.
  exportFilesList = [s for s in filesList if imageFilePrefix in s]


  # Get the list of image files and the JSON mixer file.
  imageFilesList = []
  jsonFile = None
  for f in exportFilesList:
    if f.endswith('.tfrecord.gz'):
      imageFilesList.append(f)
    elif f.endswith('.json'):
      jsonFile = f

  # Make sure the files are in the right order.
  imageFilesList.sort()

  pprint.pprint(imageFilesList)
  print(jsonFile)
  import json
  from pprint import pprint 

  # Load the contents of the mixer file to a JSON object.
  jsonText = !gsutil cat {jsonFile}
  # Get a single string w/ newlines from the IPython.utils.text.SList
  mixer = json.loads(jsonText.nlstr)
  pprint(mixer)
  # Get relevant info from the JSON mixer file.
  PATCH_WIDTH = mixer['patchDimensions'][0]
  PATCH_HEIGHT = mixer['patchDimensions'][1]
  PATCHES = mixer['totalPatches']
  PATCH_DIMENSIONS_FLAT = [PATCH_WIDTH * PATCH_HEIGHT, 1]

  bands = bandNames

  imageColumns = []

  for k in featureNames:
    imageColumns.append(tf.io.FixedLenFeature(shape=PATCH_DIMENSIONS_FLAT, dtype=tf.float32))


  # Parsing dictionary.
  imageFeaturesDict = dict(zip(bands, imageColumns))

  # Note that you can make one dataset from many files by specifying a list.
  imageDataset = tf.data.TFRecordDataset(imageFilesList, compression_type='GZIP')
  # Parse the data into tensors, one long tensor per patch.
  imageDataset = imageDataset.map(parse_image, num_parallel_calls=5)

  # Break our long tensors into many little ones.
  imageDataset = imageDataset.flat_map(
    lambda features: tf.data.Dataset.from_tensor_slices(features)
  )

  # Turn the dictionary in each record into a tuple with a dummy label.
  imageDataset = imageDataset.map(
    # Add a dummy target (-1), with a value that is obviously ridiculous.
    # This is because the model expects a tuple of (inputs, label).
    lambda dataDict: (tf.transpose(list(dataDict.values())), tf.constant(-1))
  )

  # Turn each patch into a batch.
  imageDataset = imageDataset.batch(PATCH_WIDTH * PATCH_HEIGHT)

  #imageDataset = tf.cast(imageDataset, tf.float32)
   # Run prediction in batches, with as many steps as there are patches.
  predictions = model.predict(imageDataset, steps=PATCHES, verbose=1)

  # Note that the predictions come as a numpy array.  Check the first one.
  print(predictions[0])

  outputImageFile = 'gs://' + outputBucket + '/Classified_'+ year+region+'_buff'+'_right.TFRecord'
  writer = tf.python_io.TFRecordWriter(outputImageFile)

  sess = tf.InteractiveSession()

  patch = [[]]
  curPatch = 1
  for prediction in predictions:
    patch[0].append(prediction[0][0])
    if (len(patch[0]) == PATCH_WIDTH * PATCH_HEIGHT):
      print('Done with patch ' + str(curPatch) + '...')   
      # Create an example
      example = tf.train.Example(
        features=tf.train.Features(
          feature={
              'irr_pred': tf.train.Feature(
                  float_list=tf.train.FloatList(
                      value=patch[0]))
          }))
      writer.write(example.SerializeToString())
      curPatch += 1
      patch = [[]]
      
  writer.close()
  sess.close()
  # REPLACE WITH YOUR USERNAME:
  USER_NAME = 'rappjer'
  outputAssetID = 'users/' + USER_NAME + '/'+ region +'/'+ year +'_Buff_tf_right'
  print('Writing to ' + outputAssetID)
  # Start the upload.
  !earthengine upload image --asset_id={outputAssetID} {outputImageFile} {jsonFile}

['gs://aim_nn/2007_ButlerCounty_tfRecord00000.tfrecord.gz', 'gs://aim_nn/2007_ButlerCounty_tfRecord00001.tfrecord.gz', 'gs://aim_nn/2007_ButlerCounty_tfRecord00002.tfrecord.gz', 'gs://aim_nn/2007_ButlerCounty_tfRecordmixer.json', 'gs://aim_nn/2007_Butler_buffer_tfRecord00000.tfrecord.gz', 'gs://aim_nn/2007_Butler_buffer_tfRecord00001.tfrecord.gz', 'gs://aim_nn/2007_Butler_buffer_tfRecord00002.tfrecord.gz', 'gs://aim_nn/2007_Butler_buffer_tfRecord00003.tfrecord.gz', 'gs://aim_nn/2007_Butler_buffer_tfRecord00004.tfrecord.gz', 'gs://aim_nn/2007_Butler_buffer_tfRecord00005.tfrecord.gz', 'gs://aim_nn/2007_Butler_buffer_tfRecord00006.tfrecord.gz', 'gs://aim_nn/2007_Butler_buffer_tfRecordmixer.json', 'gs://aim_nn/2007_California_buffer2_tfRecord00000.tfrecord.gz', 'gs://aim_nn/2007_California_buffer2_tfRecord00001.tfrecord.gz', 'gs://aim_nn/2007_California_buffer2_tfRecord00002.tfrecord.gz', 'gs://aim_nn/2007_California_buffer2_tfRecord00003.tfrecord.gz', 'gs://aim_nn/2007_California_buffer2_